<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/Bi-LSTM-MLP-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [4]:
y_data = data.pop('4')
x_data = data

In [5]:
train_x, test_x, train_y, test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [9]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='tanh', recurrent_activation='sigmoid', go_backwards=True, return_sequences=True), input_shape=(len(x_data.columns), 1)))
model.add(Dense(20, activation='ReLU'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='ReLU'))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 4, 200)           81600     
 l)                                                              
                                                                 
 dense (Dense)               (None, 4, 20)             4020      
                                                                 
 dropout (Dropout)           (None, 4, 20)             0         
                                                                 
 dense_1 (Dense)             (None, 4, 10)             210       
                                                                 
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 164       
                                                        

In [10]:
model.fit(train_x, train_y, batch_size=32) 

414237/414237 [==============================] - 2709s 7ms/step - loss: 0.0363 - accuracy: 0.9878


In [11]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 305s 3ms/step


In [12]:
confusion_matrix(test_y, pred_y)

array([[ 117727,       0,       0,       0],
       [      0,   94084,    4118,       0],
       [  23402,     139, 2824369,       0],
       [      0,       0,       0,  250056]])

In [13]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.83418   1.00000   0.90959    117727
           1    0.99852   0.95807   0.97788     98202
           2    0.99854   0.99173   0.99513   2847910
           3    1.00000   1.00000   1.00000    250056

    accuracy                        0.99165   3313895
   macro avg    0.95781   0.98745   0.97065   3313895
weighted avg    0.99281   0.99165   0.99195   3313895

